#               Initialize the model

## Modified the last used resnet18 model by introducing a maxpool layer after preparation layer to match TinyImagenet-200 dataset size which is 64x64
## Modification is as per ResNet 18 State-Of-The-Art Architecture

In [1]:
import torch

from model import ResNetA12 as resnet
import importlib
importlib.reload(resnet)
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
mymodel = resnet.ResNet18().to(device)
summary(mymodel, input_size=(3, 64, 64))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           1,728
       BatchNorm2d-2           [-1, 64, 64, 64]             128
         MaxPool2d-3           [-1, 64, 32, 32]               0
            Conv2d-4           [-1, 64, 32, 32]          36,864
       BatchNorm2d-5           [-1, 64, 32, 32]             128
            Conv2d-6           [-1, 64, 32, 32]          36,864
       BatchNorm2d-7           [-1, 64, 32, 32]             128
        BasicBlock-8           [-1, 64, 32, 32]               0
            Conv2d-9           [-1, 64, 32, 32]          36,864
      BatchNorm2d-10           [-1, 64, 32, 32]             128
           Conv2d-11           [-1, 64, 32, 32]          36,864
      BatchNorm2d-12           [-1, 64, 32, 32]             128
       BasicBlock-13           [-1, 64, 32, 32]               0
           Conv2d-14          [-1,

# Generate DataSet and DataLoader for Training and Testing

### 1. Set the rootpath to the path where data is stored after processing it. Check Preprocessing.ipynb
### 2. Any transformations
### 3. Generate train loader and test loader

#### Note - Custom Dataset is used here see the project structure to find TinyImagenetDataSetBuilder

In [2]:
import torch 
import torchvision
from TinyImagenetDataSetBuilder import DataSetBuilder
import torchvision.transforms as transforms

train_processed_data_path = './'
test_processed_data_path = './'
SEED = 1
# Check if CUDA is available
cuda = torch.cuda.is_available()
print("Is CUDA available: ", cuda)

torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)
    
transform = transforms.Compose(
            [transforms.ToTensor()]
        )
dataloader_args = dict(shuffle=False, batch_size=200, num_workers=0, pin_memory=True) if cuda else dict(shuffle=False, batch_size=32)
# Generators
trainset = DataSetBuilder(train_processed_data_path, transform)
trainloader = torch.utils.data.DataLoader(trainset, **dataloader_args)

testset = DataSetBuilder(test_processed_data_path, False, transform)

testloader = torch.utils.data.DataLoader(testset, **dataloader_args)

Is CUDA available:  True


# Throw the model to find Train and Test Accuracies
### 1. Using SGD optimizer
### 2. Using L2 regularization to avoid overfitting.
#### Note -  Saving the model with epoch value, model and optimizer state parameters. Helps to re-run from the interupted training point.

In [3]:
from traintest import train, test
import importlib
importlib.reload(train)
importlib.reload(test)
import torch.optim as optim

optimizer = optim.SGD(mymodel.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)

EPOCHS = 25
for epoch in range(EPOCHS):  # loop over the dataset multiple times
    print("EPOCH: ", epoch)
    train.train(mymodel, device, testloader, epoch, optimizer)
    #Save the model for future use.
    torch.save({'epoch':epoch,
                'model_state_dict': mymodel.state_dict(),
               'optimizer_state_dict': optimizer.state_dict(),
               }, 'training_a12_v2.pt')
    test.test(mymodel, device, testloader, 'l1')

  0%|          | 0/165 [00:00<?, ?it/s]

Finished Training
Finished Training
EPOCH:  0


Loss=5.044056415557861 Batch_id=164 Accuracy=97.38: 100%|██████████| 165/165 [01:17<00:00,  2.13it/s] 
  0%|          | 0/165 [00:00<?, ?it/s]


Test Set: Average loss: -17328.899993896484, Accuracy: 1606/33000 (4.87%)

EPOCH:  1


Loss=4.750633716583252 Batch_id=164 Accuracy=100.52: 100%|██████████| 165/165 [01:16<00:00,  2.17it/s] 
  0%|          | 0/165 [00:00<?, ?it/s]

...


Test Set: Average loss: -266140.5233154297, Accuracy: 23605/33000 (71.53%)

EPOCH:  23


Loss=0.8148192763328552 Batch_id=164 Accuracy=183.01: 100%|██████████| 165/165 [01:17<00:00,  2.14it/s]
  0%|          | 0/165 [00:00<?, ?it/s]


Test Set: Average loss: -283291.98571777344, Accuracy: 25023/33000 (75.83%)

EPOCH:  24


Loss=0.6072027683258057 Batch_id=164 Accuracy=186.78: 100%|██████████| 165/165 [01:19<00:00,  2.07it/s]



Test Set: Average loss: -302940.77392578125, Accuracy: 27068/33000 (82.02%)

